In [1]:
import openai
import os
openai.api_base = 'https://chatapi.a3e.top/v1'
openai.api_key = 'sk-xQCDPzIvC83disI5C7DeBa8fC2Fa4bDfB2627e5cB5603d00'
openai.__version__

'0.28.0'

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

# 情感推断

## 情感分类（正面/负面/分值）

In [3]:
# 中文
lamp_review_zh = """
我需要一盏漂亮的卧室灯，这款灯具有额外的储物功能，价格也不算太高。\
我很快就收到了它。在运输过程中，我们的灯绳断了，但是公司很乐意寄送了一个新的。\
几天后就收到了。这款灯很容易组装。我发现少了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺失的零件！\
在我看来，Lumina 是一家非常关心顾客和产品的优秀公司！
"""

In [4]:
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

积极的情感。


In [5]:
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？
请用一个词回答：正面或负面。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

正面


In [6]:
prompt = f"""
以下三个三个反引号分隔的产品评论的情感是什么？
请用一个【-10，10】之间的数字回答，10为最正面，-10为最负面。

Review text: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

8


## 识别情感类型

In [7]:
prompt = f"""
识别以下评论的作者表达的情感，不超过五个项目，将答案python list的方式返回

评论：{lamp_review_zh}
"""
response = get_completion(prompt)
print(response)

['满意', '感激', '信任', '赞扬', '满足']


## 识别愤怒/喜欢

In [8]:
prompt = f"""
以下评论的作者是否表达了愤怒？评论用三个反引号分隔。给出是或否的答案。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

否。


In [9]:
prompt = f"""
以下评论的作者是否表达了喜欢？评论用三个反引号分隔。给出是或否的答案。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

是


# 信息提取

## 商品信息提取

In [10]:
prompt = f"""
从评论文本中识别以下项目：
- 评论者购买的物品
- 制造该物品的公司

评论文本用三个反引号分隔。将你的响应格式化为以 “物品” 和 “品牌” 为键的 JSON 对象。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
  
评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

```json
{
    "物品": "卧室灯",
    "品牌": "Lumina"
}
```


## 综合信息处理

In [11]:
prompt = f"""
从评论文本中识别以下项目：
- 情绪（正面或负面）
- 审稿人是否表达了愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司

评论用三个反引号分隔。将您的响应格式化为 JSON 对象，以 “Sentiment”、“Anger”、“Item” 和 “Brand” 作为键。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
将 Anger 值格式化为布尔值。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

```json
{
    "Sentiment": "正面",
    "Anger": false,
    "Item": "卧室灯",
    "Brand": "Lumina"
}
```


# 主题推断

In [12]:
# 中文
story_zh = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。

一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太棒了。”

调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

## 推断主题

In [13]:
prompt = f"""
总结以下文本的主题。

文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

主题：公共部门员工满意度调查结果，NASA 满意度最高，社会保障管理局满意度最低，政府努力提高员工满意度。


In [14]:
prompt = f"""
总结提炼以下给定文本中的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

NASA, 员工满意度, 管理团队, 社会保障, 政府承诺


## 为特定主题制作新闻提醒

In [15]:
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以列表的形式给出答案，每个主题用 0 或 1， 比如“美国航空航天局：0”。

主题列表：美国航空航天局、当地政府、工程、员工满意度、联邦政府

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

美国航空航天局：1
当地政府：1
工程：0
员工满意度：1
联邦政府：1


In [16]:
topic_dict = {i.split('：')[0]: int(i.split('：')[1]) for i in response.split(sep='\n')}
if topic_dict['美国航空航天局'] == 1:
    print("提醒: 关于美国航空航天局的新消息")

提醒: 关于美国航空航天局的新消息
